### News

### Installation

In [3]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [4]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

!pip install -U bitsandbytes

In [ ]:
!pip install -U bitsandbytes
!pip install unsloth_zoo
!pip install blake3

In [1]:
# free up gpu memory
import torch
#del model
#del sent_transformer
torch.cuda.empty_cache()

In [2]:
from sentence_transformers import SentenceTransformer, util

from google.colab import drive
drive.mount('/content/drive')
TRAINING_DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Training Data/aidit"
n_training_examples = 30



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load a pre-trained SentenceTransformer model
sent_transformer = SentenceTransformer('all-mpnet-base-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Unsloth

In [5]:
BASE_MODEL = "unsloth/Qwen3-4B"

Load up `Qwen 2.5 3B Instruct`, and set parameters

In [6]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower


fourbit_models = [
    "unsloth/Qwen3-1.7B-unsloth-bnb-4bit", # Qwen 14B 2x faster
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [NEW] We support TTS models!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL,
    max_seq_length = 16392,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    device_map = "balanced",
    fast_inference=True,
    # token = "hf_...",      # use one if using gated models
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 7007,

)

==((====))==  Unsloth 2025.5.4: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-4b-unsloth-bnb-4bit with actual GPU utilization = 48.5%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 22.16 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 16392. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.88 GB. Also swap space = 6 GB.
INFO 05-16 19:58:36 [config.py:717] This model supports multiple tasks: {'generate', 'embed', 'score', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 05-16 19:58:36 [config.py:2003] Chunked 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-16 19:58:44 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-16 19:58:45 [gpu_model_runner.py:1347] Model loading took 3.6108 GiB and 5.153647 seconds
INFO 05-16 19:59:09 [backends.py:420] Using cache directory: /root/.cache/vllm/torch_compile_cache/b2a6c37f4b/rank_0_0 for vLLM's torch.compile
INFO 05-16 19:59:09 [backends.py:430] Dynamo bytecode transform time: 24.19 s
INFO 05-16 19:59:22 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 10.610 s
INFO 05-16 19:59:30 [monitor.py:33] torch.compile takes 24.19 s in total
INFO 05-16 19:59:31 [kv_cache_utils.py:634] GPU KV cache size: 34,960 tokens
INFO 05-16 19:59:31 [kv_cache_utils.py:637] Maximum concurrency for 16,392 tokens per request: 2.13x
INFO 05-16 20:00:48 [gpu_model_runner.py:1686] Graph capturing finished in 76 secs, took 1.42 GiB
INFO 05-16 20:00:48 [core.py:159] init engine (profile, create kv cache, warmup model) took 123.19 seconds
Unsloth: Just some info: will skip pa

Unsloth 2025.5.4 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [7]:

from transformers import pipeline
classifier = pipeline("zero-shot-classification", cache_dir=".cache")

from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

model_name = "facebook/bart-large-mnli"
classifier_tokenizer = AutoTokenizer.from_pretrained(model_name)
classification_model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("zero-shot-classification", model=classification_model, tokenizer=classifier_tokenizer)


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Device set to use cpu


In [8]:
%time
import numpy as np
sentences = ["How long have you felt this way?", "This is a statement.", "Is this a question? is this Another question?"]
results = classifier(sentences, candidate_labels=["question", "multiple questions", "other"]
                         , batch_size=len(sentences))
results = [r['labels'][np.argmax(r['scores'])] for r in results]
print(results)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs
['question', 'other', 'multiple questions']


### Data Prep & reward functions

<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [9]:
from reward_functions import thinking_reward_func, answer_reward_func

In [11]:
TRAINING_DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Training Data/aidit"
n_training_examples = 35
# load dataset from drive
dataset_path = f'{TRAINING_DATA_PATH}/dataset_multi_state_{n_training_examples}.jsonl'
dataset = load_from_disk(dataset_path)

from datasets import Features, Sequence, Value

# Define the schema explicitly
features = dataset.features.copy()
features["prompt"] = Value("string")  # Ensure it's treated as string initially

# First ensure all operations except JSON parsing work
dataset = dataset.rename_column("conversation", "prompt")
dataset = dataset.rename_column("rubric", "rubrics")
dataset = dataset.shuffle(seed=42)

df = dataset.to_pandas()

# Parse JSON in pandas which handles this operation better
def parse_json_safely(text):
    if not isinstance(text, str):
        return text
    try:
        return json.loads(text)
    except:
        return text

# Apply the parsing to the prompt column
df['prompt'] = df['prompt'].apply(parse_json_safely)

dataset = Dataset.from_pandas(df)


In [12]:
system_prompt_str = """
/think You are a helpful assistant that helps users with understanding the clinical diagnosis of their medical conditions.
In interacting with the patient try to obtain as much critical information as you can

[Thought Instructions]
* At each step determine the differential diagnosis. Do not share this with the user but use it to determine the next question to ask.
* Identify the top question to ask the member to gain higher confidence on a particular diagnosis.
* Keep track of how many questions you have asked since the start and since the last time you asked the member if they are willing to answer more quesitons.

[Response Instructions]
* Ask the member 1 question at a time. Based on the diagnosis, pick the question that will provide you most information to narrow down the diagnosis.
* Once you have asked ~40 questions, provide your best diagnosis and what extra information you might need to narrow down.
* Do not repeat questions.
* Every 10 questions or so, ask the member if they are willing to answer more questions.

Examples:
---------
User: I have a cold and fever.
Assistant: <think> A partial differential diagnosos for these symptoms: a. Cold b. Influenza c. Covid-19 d. RSV e. Pneumonia f. Bronchitis and many more. I should find out if this is acute or chronic.
This is my 5th question since the start or the last time I asked them if they are ok with answering more questions.
</think>Was the onset was sudden or gradual?
User: It was pretty sudden
Assistant: <think> A partial differential diagnosos for these symptoms: a. Cold b. Influenza c. Covid-19 d. RSV e. Pneumonia f. Bronchitis and many more. I should find out if this is acute or chronic.
This is my 6th question since the start or the last time I asked them if they are ok with answering more questions.
</think>Are you experiencing any chills or chest pain?
User: Yes

"""

# PROMPT_FILE = "prompt.txt"
# with open(PROMPT_FILE) as f:
#  system_prompt_str = json.load(f)


def apply_system_prompt(example):
    system_prompt = {"role": "system", "content": system_prompt_str}
    example['prompt'] = [system_prompt] + example['prompt'][1:]
    if len(example['prompt']) <= 2:
      example['prompt'] += [{'role': 'user', 'content':'Hi'}]
    return example

def truncate_prompt(example, max_seq_length=512):
  example['prompt'] = example['prompt'][:max_seq_length]
  return example

def last_message_user(example):
  messages = example['prompt']
  if messages[-1]['role'] == 'assistant':
    example['prompt'] = messages[:-1]
  return example

def remove_empty_content_messages(example):
  messages = example['prompt']
  for m in messages:
    if m['content'] is None or len(m['content']) == 0:
      messages.remove(m)

  example['prompt'] =messages
  return example
dataset = dataset.map(apply_system_prompt)
dataset = dataset.map(last_message_user)
dataset = dataset.map(remove_empty_content_messages)


Map:   0%|          | 0/105 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

In [13]:
import json
import numpy as np
conversation, rubric, metadata = dataset[0]['prompt'], dataset[0]['rubrics'], dataset[0]['metadata']
try:
  rubric_json = json.loads(rubric.replace("```json\n", "").replace("```", ""))
except:
  rubric_json = rubric
print(type(rubric_json))
think, answer = (None, rubric_json) if type(rubric_json) is str else (rubric_json.get('think', ""), rubric_json.get('answer', ""))
response = [{'role': 'assistant', "content": f"""<think> {think}</think>Have you been able to measure your temperature with a thermometer?"""}]
reward = thinking_reward_func(prompts=[conversation], completions=[response], rubrics=[rubric])
print(reward)
reward2 = answer_reward_func(prompts=[conversation], completions=[response], rubrics=[rubric], metadata=[metadata])
print(answer[0])
print(json.loads(rubric)['answer'])
print(metadata)
print(reward2)

<class 'dict'>
[1.00000012]
----------
Have you been able to measure your temperature with a thermometer? question 1
Have you been able to measure your temperature with a thermometer? If so, what was the reading?
['Have you been able to measure your temperature with a thermometer? If so, what was the reading?', 'Does the chest pain spread or radiate to any other areas, such as your neck, shoulders, back, or arms?', 'You mentioned sitting forward helps with your breathing and pain. Is the chest pain noticeably worse when you lie flat on your back?', 'Have you noticed any swelling, redness, warmth, or pain in your legs, particularly in your calves?', 'Have you had any recent long periods of inactivity, such as long-distance travel (e.g., a long flight or car ride), recent surgery, or extended bed rest?', 'Do you have any known pre-existing medical conditions, such as heart disease, lung conditions (like asthma or COPD), autoimmune disorders, or any history of blood clots?', 'Do you curre

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    beta = 0.001,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = 2048,
    max_completion_length = 1024,
    unsloth_num_chunks = 1,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 2


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
dataset = dataset.shuffle()
torch._dynamo.reset()
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
     thinking_reward_func,
     answer_reward_func, # Requires `rubrics` to be in batch
     # int_reward_func, # If needed
 ],
    args = training_args,
    train_dataset = dataset,
)

import os
os.environ["TORCH_LOGS"] = "+dynamo"
os.environ["TORCHDYNAMO_VERBOSE"]="1"

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 105 | Num Epochs = 4 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 132,120,576/4,000,000,000 (3.30% trained)


Unsloth: Will smartly offload gradients to save VRAM!
----------
<think>
Okay, let me try to figure out what's going on here. The user has had a persistent wet cough for about 8 months, which has been getting worse. They mentioned it's clear phlegm sometimes turning yellow or green, and they produce a lot of sputum each time they cough. They don't have blood in the sputum, and the cough comes and goes but is usually accompanied by a tickle in the throat. They have asthma but say it's not as bad as this. They don't have fever, chills, night sweats, or other symptoms like sore throat, runny nose, or nausea. They don搞笑 have traveled recently, no exposure to TB, animals, or fumes. They have type 2 diabetes and high blood pressure, controlled by diet, and take vitamin supplements. No allergies or medications, no hospitalizations, and no close contacts with ill people.

So, considering the history: chronic cough lasting 8 months, wet cough with sputum, worsening over time, associated with as

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / thinking_reward_func,rewards / answer_reward_func
1,-0.000000,-0.136740,0.463844,695.875000,0.000000,-0.285359,0.148618
2,0.000000,0.303435,0.375131,739.500000,0.000000,-0.212373,0.515809
3,0.000000,0.037659,0.683759,827.625000,0.003878,-0.402061,0.439720
4,0.000000,0.373068,0.260117,826.500000,0.001057,-0.106236,0.479305
5,0.000000,0.631381,0.192828,785.750000,0.005447,0.260870,0.370511
6,0.000000,0.330684,0.331370,833.750000,0.001154,-0.098384,0.429068
7,0.000000,0.720415,0.044877,743.375000,0.001097,0.127906,0.592509
8,0.000000,0.383806,0.254688,752.875000,0.001182,-0.007106,0.390912
9,0.000000,0.709272,0.023985,887.125000,0.000992,0.136335,0.572938
10,0.000000,0.686416,0.327178,842.500000,0.001068,0.169019,0.517397


----------
<think>
Okay, let me try to figure out what's going on here. The user has been feeling unwell for four days with symptoms like shortness of breath, chills, chest pain, and a slight runny nose. They mentioned that the chest pain is sharp and occurs when taking a deep breath, and it's constant but not getting worse. They also noted that sitting forward helps with breathing and that they've had chills for three days.

First, I need to consider possible differential diagnoses. The key symptoms here are chest pain with breathing, shortness of breath, and chills. The fact that the chest pain is sharp and occurs with deep breaths might point towards a pleural issue, like pleuritis or a pneumothorax. Alternatively, it could be a pulmonary embolism, but that's less likely given the timeline and the absence of other signs like sudden onset or chest pressure. Another possibility is a viral infection affecting the lungs, like pneumonia, which can cause chest pain and shortness of breath

In [ ]:
from datetime import datetime
dt = datetime.now()
model_size = "4B"
timestamp = dt.strftime("%Y%m%d_%H")
lora_model_name = f"qwen3_grpo_saved_lora_{model_size}_{timestamp}"
model.save_lora(f"/Content/drive/MyDrive/Colab Notebooks/models/{lora_model_name}")

In [ ]:
data = dataset['prompt'][4][0:3]
print(data)

while True:
  text = tokenizer.apply_chat_template(data,
                                      tokenize = False,
                                      add_generation_prompt = True,
                                      skip_special_tokens=True)

  from vllm import SamplingParams
  sampling_params = SamplingParams(
      temperature = 0.1,
      top_p = 0.95,
      max_tokens = 1024,
  )
  output = model.fast_generate(
      text,
      sampling_params = sampling_params,
  )[0].outputs[0].text

  print(output)
  input_= input()
  if input_ == "exit": break
  data = data + [{'role': 'assistant', 'content': output}] + [{'role': 'user', 'content': input_}]

[{'content': '\n/think You are a helpful assistant that helps users with understanding the clinical diagnosis of their medical conditions.\nIn interacting with the patient try to obtain as much critical information as you can\n\n[Thought Instructions]\n* At each step determine the differential diagnosis. Do not share this with the user but use it to determine the next question to ask.\n* Identify the top question to ask the member to gain higher confidence on a particular diagnosis.\n* Keep track of how many questions you have asked since the start and since the last time you asked the member if they are willing to answer more quesitons.\n\n[Response Instructions]\n* Ask the member 1 question at a time. Based on the diagnosis, pick the question that will provide you most information to narrow down the diagnosis.\n* Once you have asked ~40 questions, provide your best diagnosis and what extra information you might need to narrow down.\n* Do not repeat questions.\n* Every 10 questions or

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<think>
Okay, the user mentioned that this morning, after waking up, they coughed up some phlegm with blood in it and it scared them. Let me start by considering the possible causes. The presence of blood in sputum, known as hemoptysis, can be due to various conditions. Common causes include infections like bronchitis or pneumonia, but also more serious issues like tuberculosis, lung cancer, or pulmonary embolism. I need to determine if the blood is fresh or old, how much is involved, and the associated symptoms.

First, I should ask about the amount of blood. A small amount might be due to a benign condition, while a larger amount could indicate something more severe. Then, the color of the blood—bright red might suggest a recent bleed, while rust-colored could point to something else. Also, the duration of symptoms is important. If it's a new symptom, that could point to an acute issue. I should also ask about any associated symptoms like fever, chest pain, or weight loss, which migh

KeyboardInterrupt: Interrupted by user

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [16]:
data = dataset['prompt'][4][0:3]
print(data[-1])
while True:
  text = tokenizer.apply_chat_template(data,
                                      tokenize = False,
                                      add_generation_prompt = True,
                                      skip_special_tokens=True)

  from vllm import SamplingParams
  sampling_params = SamplingParams(
      temperature = 0.1,
      top_p = 0.95,
      max_tokens = 1024,
  )
  output = model.fast_generate(
      text,
      sampling_params = sampling_params,
      lora_request = model.load_lora(f"/Content/drive/MyDrive/Colab Notebooks/models/{lora_model_name}"),
  )[0].outputs[0].text

  print(output)
  input_= input()
  if input_ == "exit": break
  data = data + [{'role': 'assistant', 'content': output}] + [{'role': 'user', 'content': input_}]

{'content': 'SO I SO I HAVE COPAND ITS USUALLY LIKE PRETTY MANAGEABLE WITH MY PUFFERS AND THAT BUT JUST IN THE LAST LIKE 3 DAYS I FEEL LIKE ITS GOTTEN A LOT WORSE YOU KNOW I I FEEL LIKE IM COUGHING MORE OFTEN AND IM NOW UH KIND OF PRODUCING MORE SPUTUM KIND OF LIKE A GREEN YELLOWY SPUTUM UHM I EVEN NOTICE THAT THERE IS LIKE A LITTLE BIT OF BLOOD A COUPLE TIMES WHEN I WAS LIKE REALLY COUGHING HARD I’VE BEEN FINDING IT LIKE HARDER TO BREATHE THAN USUAL I DON’T HAVE HOME OXYGEN OR ANYTHING AND IM NOT I DIDNT THINK I WAS QUITE THERE YET BUT THIS THESE LAST FEW DAYS ITS BEEN ITS BEEN REALLY HARD TO BREATHE', 'role': 'user'}


NameError: name 'lora_model_name' is not defined

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.load_lora(f"/Content/drive/MyDrive/Colab Notebooks/models/{lora_model_name}")

OSError: Unsloth: LoRA filepath = /Content/drive/MyDrive/Colab Notebooks/models/qwen3_grpo_saved_lora_20250514s does not exist!

In [ ]:
utterance = """1 question: Have you noticed any other symptoms like a rash, swollen lymph nodes, or fatigue?
1 question: Has the child been exposed to anyone with similar symptoms recently?"""

print(question_format_reward_func(prompts=[""], completions=[[{'content': utterance}]]))
utterance2 = "Have you noticed any other symptoms like a rash, swollen lymph nodes, or fatigue?  "
print(question_format_reward_func(prompts=[""], completions=[[{'content': utterance2}]]))

1 question: Have you noticed any other symptoms like a rash, swollen lymph nodes, or fatigue?  
1 question: Has the child been exposed to anyone with similar symptoms recently?
[-0.1999999]
Have you noticed any other symptoms like a rash, swollen lymph nodes, or fatigue?  
[1.00000083]


Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

'<think>\nOkay, so the user is asking how many \'r\'s are in the word "strawberry". Let me think about this. First, I need to spell the word out. S-T-R-A-W-B-E-R-R-Y. Let me count each letter. Starting with S, then T, then R. That\'s the first \'r\'. Then A, W, B, E, R, R, Y. So after the first R, there\'s another R and another R. Wait, is that right? Let me write it down again: S-T-R-A-W-B-E-R-R-Y. So the letters are S, T, R, A, W, B, E, R, R, Y. So the letters are: positions 3 is R, then position 8 is R, and position 9 is R. Wait, no, the letters are S(1), T(2), R(3), A(4), W(5), B(6), E(7), R(8), R(9), Y(10). So there are three R\'s? Wait, but the user wrote "strawberry" and maybe there\'s a typo. Let me check again. Strawberry is spelled S-T-R-A-W-B-E-R-R-Y. So yes, after the first R, there are two more R\'s. So that\'s three R\'s. Wait, but sometimes people might miscount. Let me check each letter again. S, T, R, A, W, B, E, R, R, Y. So positions 3, 8, 9. So three R\'s. But someti

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
model.save_pretrained_merged("/content/drive/MyDrive/Colab Notebooks/models/qwen3-4B-rfted-16bit", tokenizer, save_method = "merged_16bit",)
#if False: model.push_to_hub_merged("/content/drive/MyDrive/qwen3-4B-aidad", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
#model.save_pretrained_merged("/content/drive/MyDrive/Colab Notebooks/models/qwen3-4B-aidad", tokenizer, save_method = "merged_4bit",)
#if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
#if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
#if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

Unsloth: Will remove a cached repo with size 1.4K


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 27.01 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 80.97it/s]


Unsloth: Saving tokenizer... Done.
Done.


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )